In [37]:
import os
import pandas as pd
import platform


In [38]:
## Paths Input Here
analysis = 'grammar_tables/all_patients'
if platform.uname().system == 'Darwin': #------------------------------Mac OS X---------------------------------------------------------------
    conn_path = r'/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Research/2023/rapid_dementia_screen/main_group.csv'
    out_name = os.path.basename(conn_path).split('.csv')[0]
    out_dir = os.path.join(os.path.dirname(conn_path), analysis)
    print('I have set pathnames in the Mac style')
else: #----------------------------------------------------------------Windows----------------------------------------------------------------
    conn_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_AvgR.csv'
    clin_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\patient_data\AD_Clinical_Data_CDR_ADAS_COG_13.xlsx'
    # clin_path = 'path to clinical values'
    out_dir = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\AD_to_memory_net'
    #out_dir = r'path to out dir here'
    x_roi_names = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_names.csv'
    #roi_names = '<path to roi name location>'
    print('I have set pathnames in the Windows style')
if os.path.isdir(out_dir) != True:
    os.makedirs(out_dir)
print(out_dir)

I have set pathnames in the Mac style
/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Research/2023/rapid_dementia_screen/grammar_tables/all_patients


In [39]:
sort = True

#----------------------------------------------------------------User Input Above----------------------------------------------------------------
try:
    sheet_name = 'AD_Clinical_Scores'
    alphab_cols = 'B, C, D, E, F, G, J, V'
    data_df = pd.read_excel(conn_path, sheet_name=sheet_name, usecols=alphab_cols, nrows=50)
    if sort:
        data_df.sort_values(by='Patient # CDR, ADAS', ascending=True, inplace=True)
        data_df.reset_index(inplace=True, drop=True)
        # clin_df = clin_df.set_index('Patient # CDR, ADAS')

    #Remove outlier
    outlier_index=[11, 47, 48, 49]
    data_df = data_df.drop(index=outlier_index)

    #Handle NaNs
    #Drop NANs
    # data_df.dropna(inplace=True)
    data_df.pop('Patient # CDR, ADAS')
    data_df.pop('Baseline CDR (sum of squares)')
    data_df.pop('% Change from baseline (CDR)')
except:
    data_df = pd.read_csv(conn_path)
display(data_df)

,Threshold,Accuracy,Sensitivity,Specificity,Negative Predictive Value,Positive Predictive Value
0,0.500,0.65,0.06,1.00,0.64,1.00
1,0.750,0.74,0.82,0.69,0.87,0.61
2,0.875,0.76,0.94,0.66,0.95,0.62
3,1.000,0.37,1.00,0.00,0.00,0.37


In [40]:
# sample_size_df = data_df.describe().transpose()
# display(sample_size_df)
sample_size_df = data_df
# sample_size_df.set_index(sample_size_df.columns[0], drop=True, inplace=True)
# sample_size_df

In [41]:
import pandas as pd
def trial_information(*args):
    grouping_variable = 'Trial'
    interest_variable = 'Patient # CDR, ADAS'

    ##----------------------------------------------------------------User Input Above----------------------------------------------------------------
    # Group the data by the "Trial" column and get the unique values of "Patient # CDR, ADAS"
    grouped = data_df.groupby(grouping_variable)[interest_variable].unique()

    # Write a string that describes the range of values for each trial
    grouped = grouped.apply(lambda x: [str(i) for i in x])
    grouped = grouped.apply(lambda x: "Patients " + ", ".join(x[:-1]) + " to " + x[-1] if len(x) > 1 else "Patient " + x[0])

    # Group the data by the "Trial" column and count the number of occurrences of each value
    sample_size_df = data_df.groupby("Trial").size().reset_index(name='Sample Size')
    sample_size_df["Patients"] = grouped.values
# Show the result
display(sample_size_df)

,Threshold,Accuracy,Sensitivity,Specificity,Negative Predictive Value,Positive Predictive Value
0,0.500,0.65,0.06,1.00,0.64,1.00
1,0.750,0.74,0.82,0.69,0.87,0.61
2,0.875,0.76,0.94,0.66,0.95,0.62
3,1.000,0.37,1.00,0.00,0.00,0.37


In [42]:
file_without_extention = out_dir
sample_size_df.to_latex(os.path.join(f'{file_without_extention}.tex'))

with open(f'{file_without_extention}.tex', "r") as f:
    data = f.read()

data = (r"\documentclass{standalone}" + '\n' +
        r"\usepackage{booktabs}" + '\n' +
        r"\begin{document}" + '\n' +
        data +
        r"\end{document}")

with open(f'{file_without_extention}.tex', "w") as f:
    f.write(data)


/var/folders/82/946tskyj68b6htgvndtppmz80000gp/T/ipykernel_84589/1755550286.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  sample_size_df.to_latex(os.path.join(f'{file_without_extention}.tex'))


In [43]:
import subprocess

def run_pdflatex(tex_file):
    directory = os.path.dirname(tex_file)
    print('dir:', directory)
    process = subprocess.Popen(['pdflatex', tex_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    return stdout, stderr
def run_pdfpng(file_without_extention):
    process = subprocess.run(["convert", "-density", "300", f'{file_without_extention}.pdf', f'{file_without_extention}.png'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.stdout, process.stderr
    return stdout, stderr

In [44]:
# # Convert the .tex file to a .pdf file
stdout, stderr = run_pdflatex(f'{file_without_extention}.tex')
if stderr:
    print('Error: ', stderr)
else:
    print('Output: ', stdout)

# Convert the .pdf file to a .png file
stdout, stderr = run_pdfpng(f'{os.path.basename(analysis)}')
if stderr:
    print('Error: ', stderr)
else:
    print('Output: ', stdout)
# !convert -density 300 table.pdf -quality 90 table.png

dir: /Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Research/2023/rapid_dementia_screen/grammar_tables
Output:  b'This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022/Homebrew) (preloaded format=pdflatex)\n restricted \\write18 enabled.\nentering extended mode\n\n(/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Researc\nh/2023/rapid_dementia_screen/grammar_tables/all_patients.tex\nLaTeX2e <2021-11-15> patch level 1\nL3 programming layer <2022-02-24>\n(/opt/homebrew/Cellar/texlive/20220321_4/share/texmf-dist/tex/latex/standalone/\nstandalone.cls\nDocument Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan\ndalone\n\n(/opt/homebrew/Cellar/texlive/20220321_4/share/texmf-dist/tex/latex/tools/shell\nesc.sty)\n(/opt/homebrew/Cellar/texlive/20220321_4/share/texmf-dist/tex/generic/iftex/ifl\nuatex.sty\n(/opt/homebrew/Cellar/texlive/20220321_4/share/texmf-dist/tex/generic/iftex/ift\nex.sty))\n(/opt/homebrew/Cellar/texl

In [45]:
#Move files and close
import shutil
shutil.move(os.path.basename(f'{os.path.basename(analysis)}.png'), out_dir)
shutil.move(os.path.basename(f'{os.path.basename(analysis)}.pdf'), out_dir)

'/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Research/2023/rapid_dementia_screen/grammar_tables/all_patients/all_patients.pdf'